# Raw data cleaning and analysis using Pandas, Numpy

Keywords: data cleaning, energy metrics, baseline

This notebook demonstrates the use of Python library Pandas and Numpy to clean a set of building meter data and weather data for baseline model development and validation.

# Building Energy Data Analysis
This notebook demonstrates the process of cleaning, processing, and analyzing building energy data and weather data to generate performance metrics.

## Datasets
1. **Meter Data**: Electrical power consumption data at 15-min intervals.
2. **Weather Data**: Weather observations at 15-min intervals.
3. **Site Descriptions**: Metadata about building sites.

## Objectives
- Clean and preprocess the data.
- Merge datasets for integrated analysis.
- Calculate building energy performance metrics.
- Export the metrics in JSON format.

## Imports

In [3]:
import pandas as pd
import numpy as np
import sys
import os
import json
import matplotlib.pyplot as plt


## Step 1: Load the Datasets
We load the meter data, weather data, and site descriptions for analysis.

In [8]:
# Load the data
meter_data = pd.read_csv('data/chapter2/meter-data/TwoCarnegiePlaza.csv')
weather_data = pd.read_csv('data/chapter2/SanBernadino_2018-01-01_2020-01-01_Weather.csv')
site_description = pd.read_csv('data/chapter2/sites-desc.csv')

# Display the first few rows of each dataset
meter_data.head(), weather_data.head(), site_description.head()

(      datetime           site_id  power
 0  6/1/08 0:00  TwoCarnegiePlaza  36.00
 1  6/1/08 0:15  TwoCarnegiePlaza  37.44
 2  6/1/08 0:30  TwoCarnegiePlaza  37.92
 3  6/1/08 0:45  TwoCarnegiePlaza  37.44
 4  6/1/08 1:00  TwoCarnegiePlaza  37.44,
           time  apparentTemperature  cloudCover  dewPoint  humidity  \
 0  1/1/08 0:00                 8.22         0.0  -10.5800      0.24   
 1  1/1/08 0:15                 8.34         0.0  -10.6125      0.24   
 2  1/1/08 0:30                 8.46         0.0  -10.6450      0.24   
 3  1/1/08 0:45                 8.58         0.0  -10.6775      0.24   
 4  1/1/08 1:00                 8.70         0.0  -10.7100      0.24   
 
           icon  precipIntensity  precipProbability precipType   pressure  \
 0  clear-night              0.0                0.0        NaN  1024.1900   
 1          NaN              0.0                0.0        NaN  1024.0975   
 2          NaN              0.0                0.0        NaN  1024.0050   
 3         

## Step 2: Clean and Preprocess the Data
### Meter Data
- Convert `datetime` to a proper timestamp.
- Drop rows with missing or invalid power values.

### Weather Data
- Select relevant weather attributes.
- Fill missing values with column means.

### Site Descriptions
- Ensure `site_id` values match across datasets.

In [9]:
# Clean meter data
meter_data['datetime'] = pd.to_datetime(meter_data['datetime'], errors='coerce')
meter_data = meter_data.dropna(subset=['power'])
meter_data = meter_data[meter_data['power'] >= 0]

# Clean weather data
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'], errors='coerce')
relevant_weather_columns = ['datetime', 'temperature', 'humidity', 'windSpeed', 'precipIntensity']
weather_data = weather_data[relevant_weather_columns]
weather_data = weather_data.fillna(weather_data.mean())

# Filter site descriptions
valid_site_ids = site_description['site_id'].unique()
meter_data = meter_data[meter_data['site_id'].isin(valid_site_ids)]

/var/folders/fq/6vcnqd4d3nlcwtdmsgb31hx40000gn/T/ipykernel_45411/3798752314.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  meter_data['datetime'] = pd.to_datetime(meter_data['datetime'], errors='coerce')
/var/folders/fq/6vcnqd4d3nlcwtdmsgb31hx40000gn/T/ipykernel_45411/3798752314.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_data['datetime'] = pd.to_datetime(weather_data['datetime'], errors='coerce')


## Step 3: Merge Datasets
Merge the meter data, weather data, and site descriptions for integrated analysis.

In [10]:
# Merge meter and weather data
merged_data = pd.merge_asof(
    meter_data.sort_values('datetime'), 
    weather_data.sort_values('datetime'), 
    on='datetime', 
    direction='nearest'
)

# Add site descriptions
final_data = pd.merge(merged_data, site_description, on='site_id', how='left')

## Step 4: Calculate Energy Performance Metrics
Metrics include:
- **Daily Energy Usage**: Sum of power readings per day.
- **Energy Use Intensity (EUI)**: Total energy usage per floor area.

In [12]:
# Define summer and winter seasons from utility perspective
summer_months = [6, 7, 8, 9]  # June, July, August, September
winter_months = [10, 11, 12, 1, 2, 3, 4, 5]  # Winter months

# Add columns for easier filtering
meter_data['month'] = meter_data['datetime'].dt.month
meter_data['hour'] = meter_data['datetime'].dt.hour

# Annual Electric Consumption - Summer (kWh)
summer_consumption = meter_data[meter_data['month'].isin(summer_months)]['power'].sum()

# Peak Electric Demand - Summer (kW)
summer_peak_demand = meter_data[meter_data['month'].isin(summer_months)]['power'].max()

# Peak Electric Demand - Winter (kW)
winter_peak_demand = meter_data[meter_data['month'].isin(winter_months)]['power'].max()

# Demand Threshold at the Top 50 Hours (kW)
top_50_demand_threshold = meter_data['power'].nlargest(50).min()

# Annual Electric Energy Intensity (kWh/sq.ft)
annual_consumption = meter_data['power'].sum()
site_floor_area = site_description['floor_area'].iloc[0]  # Assuming one site for simplicity
annual_eui = annual_consumption / site_floor_area

# Annual Peak Electric Demand Intensity (W/sq.ft)
annual_peak_demand = meter_data['power'].max()
peak_demand_intensity = annual_peak_demand / site_floor_area * 1000  # W/sq.ft

# Weather Sensitivity - Occupied and Unoccupied
# Occupied: Define occupied hours (e.g., 8 AM to 6 PM)
occupied_hours = meter_data[(meter_data['hour'] >= 8) & (meter_data['hour'] <= 18)]
unoccupied_hours = meter_data[(meter_data['hour'] < 8) | (meter_data['hour'] > 18)]

# Merge with weather data
occupied_weather = pd.merge_asof(occupied_hours.sort_values('datetime'), weather_data.sort_values('datetime'), on='datetime', direction='nearest')
unoccupied_weather = pd.merge_asof(unoccupied_hours.sort_values('datetime'), weather_data.sort_values('datetime'), on='datetime', direction='nearest')

# Correlation of weather sensitivity
occupied_corr = occupied_weather[['power', 'temperature']].corr().iloc[0, 1]
unoccupied_corr = unoccupied_weather[['power', 'temperature']].corr().iloc[0, 1]

# Compile metrics into a dictionary
performance_metrics = {
    "Annual Electric Consumption (Summer kWh)": summer_consumption,
    "Peak Electric Demand (Summer kW)": summer_peak_demand,
    "Peak Electric Demand (Winter kW)": winter_peak_demand,
    "Demand Threshold at Top 50 Hours (kW)": top_50_demand_threshold,
    "Annual Average Electric Energy Intensity (kWh/sq.ft)": annual_eui,
    "Annual Peak Electric Demand Intensity (W/sq.ft)": peak_demand_intensity,
    "Weather Sensitivity (Occupied)": occupied_corr,
    "Weather Sensitivity (Unoccupied)": unoccupied_corr
}

## Step 5: Export Metrics
Save the metrics as a JSON file for further use or sharing.

In [13]:
# Export metrics to a JSON file
output_json_path_metrics = 'data/chapter2/building_energy_performance_metrics.json'
with open(output_json_path_metrics, 'w') as json_file:
    json.dump(performance_metrics, json_file, indent=4)

print(f"Metrics exported to {output_json_path_metrics}")

# Display the performance metrics
performance_metrics

Metrics exported to data/chapter2/building_energy_performance_metrics.json


{'Annual Electric Consumption (Summer kWh)': 1322321.28,
 'Peak Electric Demand (Summer kW)': 314.88,
 'Peak Electric Demand (Winter kW)': nan,
 'Demand Threshold at Top 50 Hours (kW)': 278.4,
 'Annual Average Electric Energy Intensity (kWh/sq.ft)': 15.798720160577314,
 'Annual Peak Electric Demand Intensity (W/sq.ft)': 3.7620970632512125,
 'Weather Sensitivity (Occupied)': 0.28299283734436803,
 'Weather Sensitivity (Unoccupied)': 0.2258003180235979}

# Building Load Shape Analysis

## Load Shape Metrics Summary

Figure below shows several parameters that can be used to summarize load shape. The following metrics are used to characterize the load shape of a building's energy usage. These metrics provide insights into daily load patterns, peak demand behavior, and the overall energy profile of the building:

### **1. Near-Peak Load**
- **Definition**: The 97.5th percentile of the daily load.
- **Purpose**: Represents the highest levels of energy demand, helping to identify peak usage patterns and their implications for system design and capacity planning.

### **2. Near-Base Load**
- **Definition**: The 2.5th percentile of the daily load.
- **Purpose**: Reflects the minimum load during periods of low energy usage, often corresponding to off-peak times or base-level operations.

### **3. High-Load Duration**
- **Definition**: The duration (in hours) during which the load is closer to the near-peak load than to the near-base load.
- **Purpose**: Indicates how long the system operates near its peak capacity, which has implications for energy costs and equipment wear.

### **4. Rise Time**
- **Definition**: The time (in hours) between the latest time in the morning when the load is close to the near-base load and the earliest time during the day when the load exceeds halfway to the near-peak load.
- **Purpose**: Captures the speed of energy demand increase, which is critical for understanding ramp-up rates and their impact on energy systems.

### **5. Fall Time**
- **Definition**: The time (in hours) between the end of the high-load period and when the load returns to the near-base load.
- **Purpose**: Describes how quickly energy usage decreases after peak periods, which is important for assessing system responsiveness and flexibility.

These metrics collectively provide a detailed picture of energy load behavior, supporting the optimization of energy systems for efficiency, reliability, and cost-effectiveness.

![Load Shape Parameters](figures/chapter1-load-shape.png)

*Price, Phillip N. 2010. Methods for Analyzing Electric Load Shape and its Variability. Lawrence Berkeley National Laboratory, LBNL-3713E*

In [20]:
# Proceed with filtering for weekdays
meter_data['weekday'] = meter_data['datetime'].dt.weekday  # Monday = 0, Sunday = 6

# Filter for weekdays (Monday to Friday)
weekday_data = meter_data[meter_data['weekday'] < 5].copy()

# Generate an average daily load profile on weekdays
weekday_data['time'] = weekday_data['datetime'].dt.time
average_daily_profile = weekday_data.groupby('hour')['power'].mean()

# Calculate the load shape parameters
near_peak_load = np.percentile(average_daily_profile, 97.5)
near_base_load = np.percentile(average_daily_profile, 2.5)
near_peak_load_density = np.percentile(average_daily_profile, 97.5) / site_floor_area * 1000  # Near-Peak Load Density (W/sq.ft)
near_base_load_density = np.percentile(average_daily_profile, 2.5) / site_floor_area * 1000  # Near-Base Load Density (W/sq.ft)
base_to_peak_ratio = near_base_load / near_peak_load  # Base to Peak Ratio
high_load_mask = average_daily_profile >= ((near_peak_load + near_base_load) / 2)
high_load_duration = len(average_daily_profile[high_load_mask])  # High-Load Duration
base_to_peak_mask = average_daily_profile >= near_peak_load
rise_time = np.argmax(base_to_peak_mask)  # Rise Time
fall_time = len(average_daily_profile) - np.argmax(base_to_peak_mask[::-1])  # Fall Time

# Define thresholds
base_load_threshold = near_base_load * 1.1  # 10% above the near-base load
midway_to_peak = (near_peak_load + near_base_load) / 2

# Group by day for per-day analysis
weekday_data['day'] = weekday_data['datetime'].dt.date
daily_profiles = weekday_data.groupby(['day', 'hour'])['power'].mean().reset_index()

# Initialize a list to store daily rise times
daily_rise_times = []

for day, group in daily_profiles.groupby('day'):
    # Find the latest time in the morning (before 12:00 PM) close to the base load
    morning_load = group[group['hour'] < 12]
    latest_base_time = morning_load[morning_load['power'] <= base_load_threshold]['hour'].max()
    
    # Find the earliest time when the load exceeds halfway to the near-peak load
    afternoon_load = group[group['hour'] >= 12]
    earliest_midway_time = afternoon_load[afternoon_load['power'] >= midway_to_peak]['hour'].min()
    
    # Calculate rise time if both points are found
    if pd.notnull(latest_base_time) and pd.notnull(earliest_midway_time):
        daily_rise_times.append(earliest_midway_time - latest_base_time)

# Calculate the average rise time across all weekdays
average_rise_time = np.mean(daily_rise_times) if daily_rise_times else None

# Organize results into a dictionary
load_shape_parameters = {
    "Near-Peak Load (kW)": near_peak_load,
    "Near-Base Load (kW)": near_base_load,
    "Near-Peak Load Density (W/ft2)": near_peak_load_density,
    "Near-Base Load Density(W/ft2)": near_base_load_density,
    "Base to Peak Ratio": base_to_peak_ratio,
    "High-Load Duration (hours)": high_load_duration,
    "Rise Time (hours)": average_rise_time,
    "Fall Time (hours)": fall_time
}

load_shape_parameters

{'Near-Peak Load (kW)': 236.724,
 'Near-Base Load (kW)': 44.087310344827586,
 'Near-Peak Load Density (W/ft2)': 2.8283113097087145,
 'Near-Base Load Density(W/ft2)': 0.5267426980910844,
 'Base to Peak Ratio': 0.1862392927832733,
 'High-Load Duration (hours)': 11,
 'Rise Time (hours)': 8.192307692307692,
 'Fall Time (hours)': 16}